# Proportional Controller

In [ ]:
from tuto_control_lib.systems import IntroSystem
from tuto_control_lib.plot import *

import matplotlib.pyplot as plt
import numpy as np
from math import exp

We have seen that a Bang-Bang solution manages to roughly get the system in the desired state.

However, the lack of protocol and guarentees of this solution limits its adoption on production systems.

In this section, we introduce the most basic controller from Control Theory: the *Proportional Controller*.

The idea of the proportional controller is to have a response proportional to the control error.

The control error is the distance between the desired behaviour and the current state of the system.

The equation of a P Controller is the following:

$$
u(k) = K_p \times e(k) = K_p \times \left(y_{ref} - y(k)\right)
$$

where:

- $K_p$ is the propotional gain of the controller
- $y_{ref}$ is the reference value for our system (i.e. the desired value of the system output)
- $y(k)$ is the system output at iteration $k$
- $e(k)$ is the control error at iteration $k$
- $u(k)$ is the input at iteration $k$

In [ ]:
system, u_values, y_values, u, max_iter = IntroSystem(), [], [], 0, 100

reference_value = 1
kp = 3.3

for i in range(max_iter):
    y = system.sense()
    y_values.append(y)
    
    error = reference_value - y
    u = kp * error
    
    system.apply(u)
    u_values.append(u)

plot_u_y(u_values, y_values, reference_value)

As we can see, the system converges, but to a values different than the reference value.
The controller introduces oscillations before converging.

In [ ]:
print(f"Steady state value: {y_values[-1]}\nReference value: {reference_value}")

<div class="alert alert-info">
Try changing the values of the proportional gain $K_p$ and the reference value.
</div>

# Design of a Proportional Controller

To design a Proportional Controller with guarentees, we must have a model of our system.

A model, in the sense of Control Theory, is a relation between the inputs and the outputs.

The general form of a model is the following:

$$
y(k + 1) = \sum_{i = 0}^k a_i y(k - i) + \sum_{i = 0}^k b_i u(k - i)
$$

where:

- $y(k + 1)$ is the next value of the output
- $y(k-i)$ and $u(k-i)$ are previous values of the output and the input
- $a_i$ and $b_i$ are the coefficients of the model ($\forall i, (a_i, b_i) \in (\mathbb{R}, \mathbb{R})$)

Usually, and to simplify this introduction, we consider *first order models*.

This means that the model only considers the last values of $y$ and $u$ to get the next value of $y$.

$$
y(k + 1) = a y(k) + b u(k)
$$

In this section, we will suppose that we have a first order model which we know the coefficients.
In a [future section](./05_Identification.ipynb), we will look at how to find these coefficients.

In [ ]:
# Our system
system = IntroSystem()
# The coefficients
a = 0.8
b = 0.5

## Stability

The pole of the closed-loop transfer function is: $a - b K_p$.

For the closed-loop system to be stable, this pole needs to be in the unit circle: $|a - b K_p| < 1$.

Thus:

$$
\frac{a - 1}{b} < K_p < \frac{a + 1}{b}
$$

In our case:

In [ ]:
stability_lower_bound = (a - 1) / b
stability_upper_bound = (a + 1) / b
print(f"{stability_lower_bound} < K_p < {stability_upper_bound}")

## Precision

Proportional Controllers are inheritly imprecise.
But we can tune their precision ($e_{ss}$) based on the reference value ($r_{ss}$).

$$
\begin{aligned}
e_{ss} &= r_{ss} ( 1 - F_R(1)) \\
       &= r_{ss} \left(1 - \frac{b K_p}{1 - (a - b K_p)}\right) < e_{ss}^*
\end{aligned}
$$

Say we want the steady state error to be less that $e_{ss}^*$.

Then,

$$
K_p > \frac{\left(1 - \frac{e_{ss}^*}{r_{ss}}\right)\left(1 - a\right)}{b\frac{e_{ss}^*}{r_{ss}}}
$$

In our case:

In [ ]:
r_ss = 1
e_star = 0.15

precision_lower_bound = (1 - e_star/r_ss) * (1 - a)/(b * (e_star/r_ss))

print(f"K_p > {precision_lower_bound}")

## Settling Time

The settling time, or the time to reach the steady state value is defined as follows:

$$
k_s \simeq \frac{-4}{\log | a - b K_p| }
$$

Let $k_s^*$ be the desired settling time.

Then:

$$
\frac{a - \exp\left(\frac{-4}{k_s^*}\right)}{b} < K_p < \frac{a + \exp\left(\frac{-4}{k_s^*}\right)}{b}
$$

In our case:

In [ ]:
ks_star = 10
settling_time_lower_bound = (a - exp(-4/ks_star)) / b
settling_time_upper_bound = (a + exp(-4/ks_star)) / b

print(f"{settling_time_lower_bound} < K_p < {settling_time_upper_bound}")

## Maximum Overshoot

The maximum overshoot is the maximum error above the reference value.
It is defined as:

$$
M_p = | a - b K_p|
$$

If $M_p^*$ is the desired maximum overshoot, then:

$$
\frac{a - M_p^*}{b} < K_p < \frac{a + M_p^*}{b}
$$

But we really are only interested in the upper bound.

In our case:

In [ ]:
mp_star = 0.1
max_overshoot_upper_bound = (a + mp_star) / b
print(f"K_p < {max_overshoot_upper_bound}")

In [ ]:
max_y = 15
min_y = -5
fig, ax = plt.subplots()
ax.broken_barh([(stability_lower_bound, stability_upper_bound - stability_lower_bound)], (2.5, 5), facecolors='tab:blue')
ax.broken_barh([(precision_lower_bound, max_y)], (7.5, 5), facecolors='tab:red')
ax.broken_barh([(settling_time_lower_bound, settling_time_upper_bound - settling_time_lower_bound)], (12.5, 5), facecolors='tab:green')
ax.broken_barh([(min_y, max_overshoot_upper_bound - min_y)], (17.5, 5), facecolors='tab:orange')

ax.set_ylim(0, 25)
ax.set_xlim(min_y, max_y)
ax.set_xlabel('Kp')
ax.set_yticks([5, 10, 15, 20], labels=['Stability', 'Precision', 'Settling Time', 'Max. Overshoot'])
ax.grid(True)
plt.show()

As we can see, there is no value of $K_p$ that satisfies all the properties.

<div class="alert alert-danger" role="alert">
    The key point is that implementing a Proportional controller requires some <b>trade-off</b>!
</div>

In the example above, the value $K_p = 2.5$ seems to statisfy most of the properties.

In [ ]:
reference_value = 1
kp = 2.5
y_values, u_values, u, system, max_iter = [], [], 0, IntroSystem(), 20

for i in range(max_iter):
    y = system.sense()
    y_values.append(y)
    
    error = reference_value - y
    u = kp * error
    
    system.apply(u)
    u_values.append(u)

plot_u_y(u_values, y_values, reference_value)

We can observe the actual behaviour of the closed loop system and compare them to the desired behavior.

In [ ]:
e_ss = reference_value - y_values[-1]
max_overshoot = (max(y_values) - y_values[-1]) / y_values[-1]
settling_time = len([x for x in y_values if abs(x - y_values[-1]) > 0.05])

print(f"Precision: {e_ss} -> desired: < {e_star}")
print(f"Settling Time: {settling_time} -> desired: < {ks_star}")
print(f"Max. Overshoot: {max_overshoot} -> desired: < {mp_star}")

As expected, the closed loop system overshoots too much, but the other properties are respected.

<div class="alert alert-info" role="alert">
  Try to change the requirements on the closed-loop properties to find different values of $K_p$ and plot the system.
</div>

[Back to menu](./00_Main.ipynb) or [Next chapter](./04_PIController.ipynb)